In [1]:
import sys,os
import torch
import yaml
import logging
from pydantic import ValidationError

sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname(os.path.abspath(os.path.dirname(os.getcwd()))))))
from datasets.weather_bench import WeatherDataset
from models.VariableEncoder.datasets.dataset import CustomDataset
from models.VariableEncoder.training.configs import TrainingConfig
from models.VariableEncoder.training.configs import TrainingRunConfig

def get_normal_dataset(config: TrainingConfig):
    tgt_time_len = 1 * config.tgt_time_len
    device = ("cuda" if torch.cuda.is_available() else "cpu" )
    device = torch.device(device)

    weather = WeatherDataset(config.train_offset, device=device)
    # dataset.shape:  torch.Size([7309, 100, 1450])
    source, mean_std = weather.load_one(config.air_variable, config.surface_variable, config.only_input_variable, 
                                        config.constant_variable, config.train_start, config.train_end)
    dataset = CustomDataset(source, tgt_time_len, n_only_input=len(config.only_input_variable)+len(config.constant_variable))
    src_var_list = weather.get_var_code(config.air_variable, config.surface_variable + config.only_input_variable+config.constant_variable)
    tgt_var_list = weather.get_var_code(config.air_variable, config.surface_variable)
    return dataset, mean_std, (src_var_list, tgt_var_list)

config_path = os.path.join(os.path.dirname(os.getcwd()), 'configs/train_config.yaml')

try:
    with open(config_path) as f:
        config_dict = yaml.safe_load(f)
    config: TrainingRunConfig = TrainingRunConfig.parse_obj(config_dict)
except FileNotFoundError:
    logging.error(f"Config file {config_path} does not exist. Exiting.")
except yaml.YAMLError:
    logging.error(f"Config file {config_path} is not valid YAML. Exiting.")
except ValidationError as e:
    logging.error(f"Config file {config_path} is not valid. Exiting.\n{e}")


dataset, mean_std, var_list = get_normal_dataset(config.training)

데이터셋 불러오는 중...
==== LOAD DATASET ====
 <xarray.Dataset>
Dimensions:                    (time: 87649, latitude: 36, longitude: 36,
                                level: 37)
Coordinates:
  * latitude                   (latitude) float64 20.04 20.74 ... 43.95 44.65
  * level                      (level) int64 1 2 3 5 7 ... 900 925 950 975 1000
  * longitude                  (longitude) float64 115.3 116.0 ... 139.2 139.9
  * time                       (time) datetime64[ns] 2013-01-01 ... 2023-01-01
Data variables: (12/14)
    10m_u_component_of_wind    (time, latitude, longitude) float32 dask.array<chunksize=(256, 36, 36), meta=np.ndarray>
    10m_v_component_of_wind    (time, latitude, longitude) float32 dask.array<chunksize=(256, 36, 36), meta=np.ndarray>
    2m_temperature             (time, latitude, longitude) float32 dask.array<chunksize=(256, 36, 36), meta=np.ndarray>
    geopotential               (time, level, latitude, longitude) float32 dask.array<chunksize=(256, 37, 36, 36), 

100%|██████████| 12/12 [00:48<00:00,  4.01s/it]


52.71228 sec


In [2]:
%load_ext autoreload
%autoreload 2

from models.VariableEncoder.training.lightning import TrainModule

model = TrainModule.load_from_checkpoint('/workspace/Haea_dev/models/VariableEncoder/tb_logs/lightning_logs/3u8ictz8/checkpoints/epoch=4-step=21910.ckpt')

/workspace/venv/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:198: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.


In [3]:
import matplotlib.pyplot as plt
import numpy as np

def visualize(loss: torch.Tensor, title):
    print(loss.min())
    plt.plot(loss, label="rmse loss")
    plt.title(title)
    plt.show()

In [4]:
from torch.utils.data import DataLoader

data_loader = DataLoader(dataset, batch_size=16, shuffle=True, drop_last=True)

loss_set = []

for batch in data_loader:
    # loss.shape = (var_len, batch, time_len)
    loss = model(batch)
    loss_set.append(loss)


loss_set = torch.cat(loss_set, dim=1)

OutOfMemoryError: CUDA out of memory. Tried to allocate 218.00 MiB. GPU 0 has a total capacity of 23.65 GiB of which 81.69 MiB is free. Process 4168739 has 23.55 GiB memory in use. Of the allocated memory 22.66 GiB is allocated by PyTorch, and 460.84 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)